In [ ]:
from google.colab import files
uploaded = files.upload()

Saving ECTA 2023 Dataset.csv to ECTA 2023 Dataset.csv


In [ ]:
!pip install pyhopper

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 1.8 MB/s eta 0:00:00


In [ ]:
import pyhopper
import numpy as np
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import random
import time
import os

In [ ]:
df = pd.read_csv("ECTA 2023 Dataset.csv")
features = ['Relative Humidity', 'Wind Speed', 'Visibility', 'Pressure', 'Wind_Chill', 'Dewpoint Temp']
target = 'Temperature'

In [ ]:
df.dropna(inplace=True)
X = df[features].values
Y = df[target].values

In [14]:
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)
os.environ['PYTHONHASHSEED'] = str(SEED)

In [15]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=SEED)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [16]:
top_maes = []

In [17]:
def objective_function(params):
    lr = params['lr']
    n1 = int(params['n1'])
    n2 = int(params['n2'])
    bs = int(params['bs'])

    model = keras.Sequential([
        keras.layers.Input(shape=(X_train.shape[1],)),
        keras.layers.Dense(n1, activation='relu'),
        keras.layers.Dense(n2, activation='relu'),
        keras.layers.Dense(1)
    ])
    model.compile(optimizer=keras.optimizers.Adam(learning_rate=lr),
                  loss='mse', metrics=['mae'])
    model.fit(X_train, Y_train, epochs=10, batch_size=bs, verbose=0)
    loss, mae = model.evaluate(X_test, Y_test, verbose=0)

    top_maes.append(mae)
    print(f"MAE: {mae:.4f} | Params: {params}")
    return mae

In [18]:
search_space = {
    'lr': pyhopper.float(0.0005, 0.05),
    'n1': pyhopper.int(10, 100),
    'n2': pyhopper.int(10, 100),
    'bs': pyhopper.int(16, 128)
}

In [19]:
search = pyhopper.Search(search_space)
best_params = search.run(
    objective_function,
    direction="minimize",
    runtime="10m",
    n_jobs=1
)

  0%|          | [00:00<?]

Search is scheduled for 10:00 (m:s)
MAE: 0.1873 | Params: {'lr': 0.02525, 'n1': 55, 'n2': 55, 'bs': 72}
MAE: 0.0764 | Params: {'lr': 0.012584010846477419, 'n1': 69, 'n2': 13, 'bs': 26}
MAE: 0.1066 | Params: {'lr': 0.008728118299586949, 'n1': 45, 'n2': 64, 'bs': 67}
MAE: 0.1537 | Params: {'lr': 0.0005695926546847818, 'n1': 13, 'n2': 36, 'bs': 99}
MAE: 0.0787 | Params: {'lr': 0.0023422643302840242, 'n1': 54, 'n2': 42, 'bs': 26}
MAE: 0.1240 | Params: {'lr': 0.017932178905253386, 'n1': 69, 'n2': 18, 'bs': 45}
MAE: 0.1437 | Params: {'lr': 0.025491824415983164, 'n1': 73, 'n2': 13, 'bs': 26}
MAE: 0.0801 | Params: {'lr': 0.012584010846477419, 'n1': 69, 'n2': 32, 'bs': 52}
MAE: 0.0867 | Params: {'lr': 0.007501841459051498, 'n1': 69, 'n2': 13, 'bs': 50}
MAE: 0.0925 | Params: {'lr': 0.012584010846477419, 'n1': 55, 'n2': 19, 'bs': 26}
MAE: 0.0857 | Params: {'lr': 0.012584010846477419, 'n1': 71, 'n2': 13, 'bs': 26}
==================== Summary ====================
Mode              : Best f  : Step

In [20]:
print("\n Best configuration found:")
print(best_params)
final_mae = objective_function(best_params)
print(f"Final MAE: {final_mae:.4f}")


 Best configuration found:
{'lr': 0.012584010846477419, 'n1': 69, 'n2': 13, 'bs': 26}
MAE: 0.0825 | Params: {'lr': 0.012584010846477419, 'n1': 69, 'n2': 13, 'bs': 26}
Final MAE: 0.0825


In [21]:
# Show top MAEs during the search
top_maes.sort()
print("\n Top 5 lowest MAEs found during search:")
for i, val in enumerate(top_maes[:5]):
    print(f"{i+1}. MAE = {val:.4f}")


 Top 5 lowest MAEs found during search:
1. MAE = 0.0764
2. MAE = 0.0787
3. MAE = 0.0801
4. MAE = 0.0825
5. MAE = 0.0857
